## Introduction to task based

Now that the data is clean we can perform analysis. Resting state analysis is completed in notebook 6. This is a review of the general linear model using data that was collected during a breathholding task. 

### Step 1: Ensure you have data from the breath-hold task downloaded to your local machine. 
    
Run the following command in terminal to grab all the fmriprep func data for the sub-10788.

```
# Download unprocessed data
cat download_list |   xargs -I '{}' aws s3 sync --no-sign-request   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/{}/func   ./data/ds000030/{}/func   --exclude '*'   --include '*sub-10788*' 

# Download anatomical data
cat download_list |   xargs -I '{}' aws s3 sync --no-sign-request   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/anat   ./data/ds000030/derivatives/fmriprep/{}/anat   --exclude '*'   --include '*sub-10788*'

# Download functional data - breath hold task
cat download_list |   xargs -I '{}' aws s3 sync --no-sign-request   s3://openneuro/ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{}/func   ./data/ds000030/derivatives/fmriprep/{}/func   --exclude '*'   --include '*sub-10788*task-bht_bold*'

```
    
### Step 2: We need to figure out the stimulus for the breath holding task. 

The description of this task is contained in `data/ds000030/task-bht_bold.json`. This does not give precise timing but luckily those are included in the table below:

| Parameter     |  Time (Seconds)      | 
| :-------------: | :-------------: | 
| Rest Time     | 16.5 |
| Activation Time     | 13.5 |
| Block Length     | 30 |
| Scan Time | 150      |
| Repetition Time | 2      |

### Step 3: Now that we know what stimulus was used we need to convolve it with a hemodynamic response function. 

The hemodynamic response function (HRF) represents the canconical BOLD response to a delta stimulus. This must be convolved with our task in order to obtain a good activity estimate. This is due to the lag in the BOLD response as it is a vascular response based on increases in cerebral blood volume and cerebral blood flow. The canonical HRF can be obtained with the nipy.modalities.fmri.hrf.spmt function. 

This HRF response is in TR just like of `task_regressor`. We can now convolve and plot them using np.convolve(). Convolution represents the task after it is sampled by the HRF as we assume the BOLD response is a linear time invariant system.

### Step 4: Now can we fit the fmri data 

In order to determine how much each voxel in the brain was responding to the task we need to perform linear regression between the task response (aka the convolved timeseries) and the data. This regression can be performed a multitude of ways taking into account things like physiology regressors, derivatives, or auto-correlation functions. In the simplest case this regression is just determining how much each voxel responded to the task response and expressing this as $\beta_1$ the mean intensity of that voxel is set to $\beta_0$. 

$Y = \beta_1 X + \beta_0 + \epsilon$

Let's do the simplest example, least squares regression, and look at the results. We want to perform a linear regression between our task response and every voxel's timeseries in our region of interest (aka the brain). First we must load in and mask the data as well as transform it to a 2D array.

**EXERCISE:** Mask and display the functional data.

We also want to transform our data to a shape we can use in our fit. This can be done using the `NiftiMasker` function `fit_transform`.

We must put our task response and our intercept into a matrix called the design matrix. 

Least squares regression can be performed by np.linalg.lstsq.

There are many ways to normalize the voxel responses but we will look at two: percent signal change and the t-statistic.

Percent signal change measures how much the voxel responded relative to the voxel mean and is calulated as shown:

$\% Signal Change=\frac{100\beta_1}{\beta_0}$

t-statistic normalizes the data by the error in the residuals and represents how strong the fit is. It is calulated as shown:

$t-statistic=\frac{\beta_1}{std(\epsilon)}$

### Step 5: Visualizing fit results

Now to visualize the results of our fit on a single voxel level. Lets look at a plot of voxel with the highest t-stat and the fit response as well as the orginal data.

We can also look at the residuals across space and time. These can show issues such as motion or physiology contamination.

### Step 6: Improving our fit with confound regressors 

The least squares fit we ran before did not take into account temporal shifts in the hemodynamic response that may occur due to physiological variance. We can use the derivative of our response as an extra regressor to help compensate for this. This is due to the first order Taylor approximation that $f(x)=f(x-a)+f'(x-a)(x-a)$. Lets add this to our design matrix as a confound.

The other confounds we can include come from fMRIprep. This is identical to notebook 5. Let's apply this to task data below.

Unlike in resting state we want to regress these out during our task fit. So we will use OLS as normal.

**EXERCISE:** Perform and plot the OLS with the new design matrix.

**EXERCISE:** Plot fit results for a single voxel.

### Step 7: Wrapping up and Final Thoughts

There are a million ways you can fit functional data and we have only shown a few here. Neuroimaging packages will include things like autocorrelation estimation and removal as well as multiple comparison correction. This excercise was to get a feel for how fMRI regression works and what changing our design matrix does to our data.